<a href="https://colab.research.google.com/github/crystdang/DABC-Final-G5/blob/Crystina/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-23 22:34:43--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.5’

postgresql-42.2.16. 100%[===================>] 979.38K  5.10MB/s    in 0.2s    

2023-01-23 22:34:43 (5.10 MB/s) - ‘postgresql-42.2.16.jar.5’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://uoft-dabc-final-g5.s3.us-east-2.amazonaws.com/Wellbeing_and_lifestyle_data_Kaggle.csv"
spark.sparkContext.addFile(url)
data_df = spark.read.csv(SparkFiles.get("Wellbeing_and_lifestyle_data_Kaggle.csv"), sep=",", header=True, inferSchema=True)

In [5]:
data_df.show()

+---------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|Timestamp|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+---------+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+
|   7/7/15|        

In [6]:
data_df.printSchema()

root
 |-- Timestamp: string (nullable = true)
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- DAILY_STRESS: string (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GE

In [7]:
data_df.columns

['Timestamp',
 'FRUITS_VEGGIES',
 'DAILY_STRESS',
 'PLACES_VISITED',
 'CORE_CIRCLE',
 'SUPPORTING_OTHERS',
 'SOCIAL_NETWORK',
 'ACHIEVEMENT',
 'DONATION',
 'BMI_RANGE',
 'TODO_COMPLETED',
 'FLOW',
 'DAILY_STEPS',
 'LIVE_VISION',
 'SLEEP_HOURS',
 'LOST_VACATION',
 'DAILY_SHOUTING',
 'SUFFICIENT_INCOME',
 'PERSONAL_AWARDS',
 'TIME_FOR_PASSION',
 'WEEKLY_MEDITATION',
 'AGE',
 'GENDER',
 'WORK_LIFE_BALANCE_SCORE']

In [8]:
data_df.describe().show()

+-------+-----------+------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+------------+------+-----------------------+
|summary|  Timestamp|    FRUITS_VEGGIES|      DAILY_STRESS|    PLACES_VISITED|       CORE_CIRCLE|SUPPORTING_OTHERS|   SOCIAL_NETWORK|       ACHIEVEMENT|          DONATION|         BMI_RANGE|   TODO_COMPLETED|              FLOW|       DAILY_STEPS|      LIVE_VISION|       SLEEP_HOURS|     LOST_VACATION|    DAILY_SHOUTING|  SUFFICIENT_INCOME|   PERSONAL_AWARDS|  TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|
+-------+-----------+------------------+------------------+------------------+------------------+-----------------+-

In [9]:
data_df.groupBy("Timestamp").count().show()

+-------------+-----+
|    Timestamp|count|
+-------------+-----+
|      9/29/15|   12|
|      1/22/16|    6|
|      10/4/17|   19|
|      7/15/18|    7|
|1/30/19 12:29|    1|
|2/26/19 15:43|    1|
| 3/14/19 6:15|    1|
| 4/20/19 3:51|    1|
| 4/25/19 8:04|    1|
|5/12/19 13:18|    1|
| 5/22/19 2:24|    1|
|5/26/19 13:40|    1|
|7/25/19 17:41|    1|
| 8/31/19 4:08|    1|
|9/20/19 17:42|    1|
| 11/3/19 7:45|    1|
|11/7/19 20:27|    1|
|11/9/19 17:44|    1|
| 1/18/20 7:59|    2|
|2/18/20 18:41|    1|
+-------------+-----+
only showing top 20 rows



In [10]:
# REFERENCE: https://sparkbyexamples.com/pyspark/pyspark-cast-column-type/
data_df = data_df.withColumn("DATE", data_df["Timestamp"].cast("date")).drop("Timestamp")
data_df.printSchema()

root
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- DAILY_STRESS: string (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- WORK_

In [11]:
from pyspark.sql.functions import year, month
data_df.select(year(data_df["DATE"])).show()

+----------+
|year(DATE)|
+----------+
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
|      null|
+----------+
only showing top 20 rows



In [12]:
data_df.groupBy("AGE").count().show()

+------------+-----+
|         AGE|count|
+------------+-----+
|  51 or more| 3390|
|    36 to 50| 4655|
|    21 to 35| 6108|
|Less than 20| 1819|
+------------+-----+



In [13]:
high_low = data_df.orderBy(data_df["WORK_LIFE_BALANCE_SCORE"].desc())
high_low.show()

+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+----+
|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|DATE|
+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+----+
|             5|           0

In [14]:
low_high = data_df.orderBy(data_df["WORK_LIFE_BALANCE_SCORE"])
low_high.show()

+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+----+
|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|DATE|
+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+----+
|             0|           5

In [15]:
from pyspark.sql.functions import avg
data_df.select(avg("WORK_LIFE_BALANCE_SCORE")).show()

+----------------------------+
|avg(WORK_LIFE_BALANCE_SCORE)|
+----------------------------+
|           666.7515026296032|
+----------------------------+



In [16]:
female_df = data_df.filter(data_df["GENDER"] == "Female")
female_df.show()

+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+----+
|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|         AGE|GENDER|WORK_LIFE_BALANCE_SCORE|DATE|
+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+------------+------+-----------------------+----+
|             3|           2

In [17]:
male_df = data_df.filter(data_df["GENDER"] == "Male")
male_df.show()

+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+----+
|FRUITS_VEGGIES|DAILY_STRESS|PLACES_VISITED|CORE_CIRCLE|SUPPORTING_OTHERS|SOCIAL_NETWORK|ACHIEVEMENT|DONATION|BMI_RANGE|TODO_COMPLETED|FLOW|DAILY_STEPS|LIVE_VISION|SLEEP_HOURS|LOST_VACATION|DAILY_SHOUTING|SUFFICIENT_INCOME|PERSONAL_AWARDS|TIME_FOR_PASSION|WEEKLY_MEDITATION|       AGE|GENDER|WORK_LIFE_BALANCE_SCORE|DATE|
+--------------+------------+--------------+-----------+-----------------+--------------+-----------+--------+---------+--------------+----+-----------+-----------+-----------+-------------+--------------+-----------------+---------------+----------------+-----------------+----------+------+-----------------------+----+
|             4|           2|     

In [18]:
data_df.groupby("DAILY_STRESS").count().show()

+------------+-----+
|DAILY_STRESS|count|
+------------+-----+
|           3| 4398|
|           0|  676|
|           5| 2052|
|      1/1/00|    1|
|           1| 2478|
|           4| 2960|
|           2| 3407|
+------------+-----+



In [19]:
clean_df = data_df.filter(data_df.DAILY_STRESS <= 5)
clean_df.groupBy("DAILY_STRESS").count().show()

+------------+-----+
|DAILY_STRESS|count|
+------------+-----+
|           3| 4398|
|           0|  676|
|           5| 2052|
|           1| 2478|
|           4| 2960|
|           2| 3407|
+------------+-----+



In [20]:
clean_df = clean_df.withColumn("DAILY_STRESS_clean", clean_df["DAILY_STRESS"].cast('int')).drop("DAILY_STRESS").withColumnRenamed("DAILY_STRESS_clean", "DAILY_STRESS")
clean_df.printSchema()

root
 |-- FRUITS_VEGGIES: integer (nullable = true)
 |-- PLACES_VISITED: integer (nullable = true)
 |-- CORE_CIRCLE: integer (nullable = true)
 |-- SUPPORTING_OTHERS: integer (nullable = true)
 |-- SOCIAL_NETWORK: integer (nullable = true)
 |-- ACHIEVEMENT: integer (nullable = true)
 |-- DONATION: integer (nullable = true)
 |-- BMI_RANGE: integer (nullable = true)
 |-- TODO_COMPLETED: integer (nullable = true)
 |-- FLOW: integer (nullable = true)
 |-- DAILY_STEPS: integer (nullable = true)
 |-- LIVE_VISION: integer (nullable = true)
 |-- SLEEP_HOURS: integer (nullable = true)
 |-- LOST_VACATION: integer (nullable = true)
 |-- DAILY_SHOUTING: integer (nullable = true)
 |-- SUFFICIENT_INCOME: integer (nullable = true)
 |-- PERSONAL_AWARDS: integer (nullable = true)
 |-- TIME_FOR_PASSION: integer (nullable = true)
 |-- WEEKLY_MEDITATION: integer (nullable = true)
 |-- AGE: string (nullable = true)
 |-- GENDER: string (nullable = true)
 |-- WORK_LIFE_BALANCE_SCORE: double (nullable = true)

In [21]:
import pandas as pd
pandas_df = data_df.toPandas()

In [22]:
pandas_df.head(10)

,FRUITS_VEGGIES,DAILY_STRESS,PLACES_VISITED,CORE_CIRCLE,SUPPORTING_OTHERS,SOCIAL_NETWORK,ACHIEVEMENT,DONATION,BMI_RANGE,TODO_COMPLETED,...,LOST_VACATION,DAILY_SHOUTING,SUFFICIENT_INCOME,PERSONAL_AWARDS,TIME_FOR_PASSION,WEEKLY_MEDITATION,AGE,GENDER,WORK_LIFE_BALANCE_SCORE,DATE
0,3,2,2,5,0,5,2,0,1,6,...,5,5,1,4,0,5,36 to 50,Female,609.5,None
1,2,3,4,3,8,10,5,2,2,5,...,2,2,2,3,2,6,36 to 50,Female,655.6,None
2,2,3,3,4,4,10,3,2,2,2,...,10,2,2,4,8,3,36 to 50,Female,631.6,None
3,3,3,10,3,10,7,2,5,2,3,...,7,5,1,5,2,0,51 or more,Female,622.7,None
4,5,1,3,3,10,4,2,4,2,5,...,0,0,2,8,1,5,51 or more,Female,663.9,None
5,3,2,3,9,10,10,2,3,1,6,...,0,2,2,10,8,3,51 or more,Female,722.3,None
6,4,2,10,6,10,10,3,5,2,8,...,10,0,2,10,8,10,51 or more,Male,727.2,None
7,3,4,5,3,5,7,4,0,1,8,...,0,2,2,8,2,2,21 to 35,Female,676.2,None
8,5,3,6,4,3,3,5,4,1,10,...,0,2,2,10,3,10,21 to 35,Female,702.8,None
9,4,4,2,6,10,10,0,4,2,3,...,0,0,1,3,8,1,51 or more,Female,634.4,None


In [23]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dabc-final-g5.cnjpfxvbwyqz.us-east-1.rds.amazonaws.com:5432/dabc-final-g5"
config = {"user":"postgres", 
          "password": "DataAnalytics", 
          "driver":"org.postgresql.Driver"}


In [24]:
# Write DataFrame to lifestyle_wellbeing table in RDS

# data_df.write.jdbc(url=jdbc_url, table='active_user', mode=mode, properties=config)